# **INTRODUCTIONS**

---
This source addresses isue COR-1173, concerning a change in agegroups in a RIVM-file. In the new format the agegroup 0-14 is detailed into the agegroups 0-4, 5-9 and 10-14. In our output, we only distinguish 0-19, so all we need to do here is add a few lines in the Stored Procedure that fills the table in VWSDEST so it recodes all three new agegroups to 0-19. 

The source code contains a single section:

3. **[Stored Procedures](#stored-procedures)**


In [ ]:
-- Copyright (c) 2021 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER   PROCEDURE [dbo].[SP_RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP_DEST]
AS
BEGIN			
WITH BASE_CTE AS (
    SELECT 
	   [DATE_OF_REPORT]
      ,[DATE_OF_STATISTICS_WEEK_START]
      ,CASE 
      	WHEN [AGE_GROUP] = '0-4'  THEN '0-19'
	    WHEN [AGE_GROUP] = '5-9'  THEN '0-19'
	    WHEN [AGE_GROUP] = '10-14'  THEN '0-19'
		WHEN [AGE_GROUP] = '0-14'  THEN '0-19'
        WHEN [AGE_GROUP] = '15-19'  THEN '0-19'        
		WHEN [AGE_GROUP] = '20-24' THEN '20-29'
		WHEN [AGE_GROUP] = '25-29' THEN '20-29'
		WHEN [AGE_GROUP] = '30-34' THEN '30-39'	
		WHEN [AGE_GROUP] = '35-39' THEN '30-39'	
		WHEN [AGE_GROUP] = '40-44' THEN '40-49'
		WHEN [AGE_GROUP] = '45-49' THEN '40-49'	
		WHEN [AGE_GROUP] = '50-54' THEN '50-59'
		WHEN [AGE_GROUP] = '55-59' THEN '50-59'
		WHEN [AGE_GROUP] = '60-64' THEN '60-69'
		WHEN [AGE_GROUP] = '65-69' THEN '60-69'
		WHEN [AGE_GROUP] = '70-74' THEN '70-79'
		WHEN [AGE_GROUP] = '75-79' THEN '70-79'
		WHEN [AGE_GROUP] = '80-84' THEN '80-89'
		WHEN [AGE_GROUP] = '85-89' THEN '80-89'
		WHEN [AGE_GROUP] = '90+'   THEN '90+'
		ELSE [AGE_GROUP]
		END AS [AGEGROUP]
      ,[HOSPITAL_ADMISSION_NOTIFICATION]
      ,[HOSPITAL_ADMISSION]
      ,[IC_ADMISSION_NOTIFICATION]
      ,[IC_ADMISSION] 
    FROM VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP)
)
,TOTALS_CTE AS (
-- Totals per agegroup per date
SELECT 
	 [DATE_OF_REPORT]								
	,[DATE_OF_STATISTICS_WEEK_START]
	,[AGEGROUP]
    ,SUM([HOSPITAL_ADMISSION_NOTIFICATION])				AS [HOSPITAL_ADMISSION_NOTIFICATION]
    ,SUM([HOSPITAL_ADMISSION])							AS [HOSPITAL_ADMISSION]
    ,SUM([IC_ADMISSION_NOTIFICATION])					AS [IC_ADMISSION_NOTIFICATION]
    ,SUM([IC_ADMISSION])								AS [IC_ADMISSION]
	FROM BASE_CTE
	GROUP BY [DATE_OF_REPORT],[DATE_OF_STATISTICS_WEEK_START],[AGEGROUP]
UNION ALL 
-- Totals per date (values per agegroup summed)
SELECT 
	 [DATE_OF_REPORT]								
	,[DATE_OF_STATISTICS_WEEK_START]
	,'total'                                            AS [AGEGROUP]
    ,SUM([HOSPITAL_ADMISSION_NOTIFICATION])				AS [HOSPITAL_ADMISSION_NOTIFICATION]
    ,SUM([HOSPITAL_ADMISSION])							AS [HOSPITAL_ADMISSION]
    ,SUM([IC_ADMISSION_NOTIFICATION])					AS [IC_ADMISSION_NOTIFICATION]
    ,SUM([IC_ADMISSION])								AS [IC_ADMISSION]
	FROM BASE_CTE
	GROUP BY [DATE_OF_REPORT],[DATE_OF_STATISTICS_WEEK_START]
),
AGEGROUP_POPULATION AS (
SELECT 
	 AGEGROUP
	,DATUM_PEILING
	,POPULATIE
FROM VWSSTATIC.CBS_POPULATION_AGEGROUP_HOSPITAL_IC
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_AGEGROUP_HOSPITAL_IC)
UNION ALL 
SELECT
	   'Total' AS [AGEGROUP]
      ,[DATUM_PEILING]
      ,[POPULATIE]
FROM [VWSSTATIC].[CBS_POPULATION_NL]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_NL)
)
,AGEGROUP_POPULATION_DATERANGE AS (
SELECT 
	 AGEGROUP
	,DATUM_PEILING AS DATE_FROM
    -- 2100-01-01 is used as convention for the most recent versions as no end date is known yet.
    ,ISNULL(LEAD(DATUM_PEILING,1) OVER (PARTITION BY AGEGROUP ORDER BY [DATUM_PEILING] ASC),CAST('2100-01-01' AS DATE)) AS DATE_TO
	,CAST(POPULATIE AS FLOAT) AS [POPULATION]
FROM AGEGROUP_POPULATION 
)
,
CALCULATED_CTE AS (
	SELECT
         [DATE_OF_REPORT]									AS [DATE_OF_REPORT]
        ,[DATE_OF_STATISTICS_WEEK_START]					AS [DATE_OF_STATISTICS_WEEK_START]
		,DATEADD(day,6,[DATE_OF_STATISTICS_WEEK_START])		AS [DATE_OF_STATISTICS_WEEK_END]		
        ,T0.AGEGROUP										AS [AGE_GROUP]
        ,[HOSPITAL_ADMISSION_NOTIFICATION]					AS [HOSPITAL_ADMISSION_NOTIFICATION]
        ,[HOSPITAL_ADMISSION]								AS [HOSPITAL_ADMISSION]
        ,[IC_ADMISSION_NOTIFICATION]						AS [IC_ADMISSION_NOTIFICATION]
        ,[IC_ADMISSION]										AS [IC_ADMISSION]
		,(1000000 / [POPULATION]) * [HOSPITAL_ADMISSION]	AS [HOSPITAL_ADMISSION_PER1M]
		,(1000000 / [POPULATION]) * [IC_ADMISSION]			AS [IC_ADMISSION_PER1M]

		--Additional data
		,ROW_NUMBER() OVER (PARTITION BY T0.AGEGROUP ORDER BY [DATE_OF_STATISTICS_WEEK_START] DESC ) AS LAST_WEEK_NR
		,T1.[POPULATION]
    FROM  TOTALS_CTE T0
    INNER JOIN AGEGROUP_POPULATION_DATERANGE T1 --only use known dateranges, neglect 'Unknown'
        ON  T0.AGEGROUP = T1.AGEGROUP
		-- Use population numbers at start of weekrange
        AND (T0.DATE_OF_STATISTICS_WEEK_START >= T1.DATE_FROM AND T0.DATE_OF_STATISTICS_WEEK_START < T1.DATE_TO)
)
--Determine ranges in which population numbers should be used
	INSERT INTO VWSDEST.RIVM_HOSPITAL_IC_ADMISSIONS_OVERTIME_BYAGEGROUP (
		[DATE_OF_REPORT]
		,[DATE_OF_STATISTICS_WEEK_START]
		,[DATE_OF_STATISTICS_WEEK_END]
		,[AGE_GROUP_GROUPED]
		,[HOSPITAL_ADMISSION_NOTIFICATION]
		,[HOSPITAL_ADMISSION]
		,[IC_ADMISSION_NOTIFICATION]
		,[IC_ADMISSION]
		,[HOSPITAL_ADMISSION_PER1M]
		,[IC_ADMISSION_PER1M]		
		)

SELECT 
         [DATE_OF_REPORT]
        ,[DATE_OF_STATISTICS_WEEK_START]
		,[DATE_OF_STATISTICS_WEEK_END]		
        ,[AGE_GROUP]
        ,[HOSPITAL_ADMISSION_NOTIFICATION]
        ,[HOSPITAL_ADMISSION]
        ,[IC_ADMISSION_NOTIFICATION]
        ,[IC_ADMISSION]
		,CAST([HOSPITAL_ADMISSION_PER1M]	as decimal(19,1))	AS [HOSPITAL_ADMISSION_PER1M]
		,CAST([IC_ADMISSION_PER1M]			as decimal(19,1))	AS [IC_ADMISSION_PER1M]
	FROM CALCULATED_CTE
	WHERE -- Remove records from last week if they contain NULLS
		LAST_WEEK_NR > 1 OR ([HOSPITAL_ADMISSION] IS NOT NULL AND IC_ADMISSION IS NOT NULL)
	ORDER BY DATE_OF_STATISTICS_WEEK_START, AGE_GROUP

END;
GO
